In [2]:
!pip install praw mlflow kserve tenacity -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
feast 0.49.0 requires uvicorn[standard]==0.34.0, but you have uvicorn 0.30.6 which is incompatible.


In [3]:
import kfp
import os


from kfp.dsl import Input, component
from kfp.dsl import OutputPath, pipeline
client = kfp.Client()

/opt/conda/lib/python3.11/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


In [4]:

@component(
    base_image="python:3.11",
    packages_to_install=["praw"]
)
def download_dataset(dataset_path: OutputPath('Dataset')) -> None:
    import praw
    from pathlib import Path
    import json
    import os
    reddit = praw.Reddit(
        client_id=os.getenv('REDDIT_CLIENT_ID'),
        client_secret=os.getenv('REDDIT_CLIENT_PW'),
        user_agent="YOUR_USER_AGENT",
        # Optional: For authenticated requests
        username=os.getenv('REDDIT_USER'),
        password=os.getenv('REDDIT_PW')
    )
    subreddit_names = [
        "funny",
        "AskReddit",
        "gaming",
        "worldnews",
        "todayilearned",
        "Music",
        "aww",
        "movies",
        "memes",
        "science"
    ]
    data_path = Path(dataset_path)
    data_path.mkdir(exist_ok=True)

    for subreddit_name in subreddit_names:
        print(subreddit_name)
        subreddit = reddit.subreddit(subreddit_name)
        for submission in subreddit.top(time_filter="day", limit=10):
        
            most_upvoted_comment = None
            highest_score = -1
            
            for comment in submission.comments.list():
                if not hasattr(comment, "author") or not comment.author:  # Skip deleted comments
                    continue
                if comment.score > highest_score:
                    highest_score = comment.score
                    most_upvoted_comment = comment
            text = submission.selftext_html
            if most_upvoted_comment is not None:
                comment_text = most_upvoted_comment.body.strip()
                comment_score = most_upvoted_comment.score
            else:
                comment_text = ""
                comment_score = 0
            url = submission.url
            title = submission.title
            id  = submission.id
            print("\tid")
            submission_data = {
                "id": id,
                "title": title,
                "url": url,
                "text": text,
                "top_comment": comment_text,
                "comment_score": comment_score
            }
            with (data_path / f"{id}.json").open("w+") as f:
                json.dump(submission_data, f)

# @component(
#     base_image="python:3.11",
#     packages_to_install=["pandas==2.2.2", "pyarrow==15.0.2"]
# )
# def preprocess_dataset(dataset: InputPath('Dataset'), output_file: OutputPath('Dataset')) -> None:
#     import pandas as pd

#     df = pd.read_csv(dataset, header=0)
#     df.columns = [c.lower().replace(" ", "_") for c in df.columns]
#     df.to_parquet(output_file)
    

In [56]:
ISVC_NAME = "reddit-model"
MLFLOW_RUN_NAME = "reddit_models"
MLFLOW_MODEL_NAME = "reddit-model"

# mlflow_tracking_uri = os.getenv('MLFLOW_TRACKING_URI')
# mlflow_s3_endpoint_url = os.getenv('MLFLOW_S3_ENDPOINT_URL')
# aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
# aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

client_id=os.getenv('REDDIT_CLIENT_ID')
client_secret=os.getenv('REDDIT_CLIENT_PW')
# Optional: For authenticated requests
username=os.getenv('REDDIT_USER')
password=os.getenv('REDDIT_PW')

@pipeline(name='download-reddit')
def download_preprocess_train_deploy_pipeline():
    download_task = download_dataset(
    ).set_env_variable(name='REDDIT_CLIENT_ID', value=client_id) \
    .set_env_variable(name='REDDIT_CLIENT_PW', value=client_secret) \
    .set_env_variable(name='REDDIT_USER', value=username) \
    .set_env_variable(name='REDDIT_PW', value=password)
    print(download_task)

In [57]:
# run = client.create_run_from_pipeline_func(download_preprocess_train_deploy_pipeline, arguments={}, enable_caching=False)

In [58]:
# kfp.compiler.Compiler().compile(download_preprocess_train_deploy_pipeline, package_path='pipeline.yaml')
# client.upload_pipeline("pipeline.yaml", 'download-reddit')

{'created_at': datetime.datetime(2025, 8, 20, 4, 59, 45, tzinfo=tzlocal()),
 'description': None,
 'display_name': 'download-reddit',
 'error': None,
 'namespace': None,
 'pipeline_id': 'f98b6779-0e3c-4ca3-a356-68f52f2315ef'}

In [12]:
from kfp import dsl
from kfp.dsl import Dataset

from kfp.v2.dsl import importer

@dsl.component
def consume_dataset(input_dataset: Input[Dataset]):
    from pathlib import Path
    files = list(Path(input_dataset.path).glob("*"))
    print("Num Files:", len(files))
    # Show file names
    for file in files:
        print(file)
    

@dsl.pipeline(name='data-consumer-pipeline')
def data_consumer_pipeline(dataset_uri: str):
    # Import the dataset artifact from the specified URI
    imported_dataset_task = importer(
        artifact_uri=dataset_uri,
        artifact_class=Dataset,
        reimport=False # Set to True if you want a new ML Metadata entry
    )
    consume_dataset(input_dataset=imported_dataset_task.outputs['artifact'])

In [13]:
run = client.create_run_from_pipeline_func(
    data_consumer_pipeline, 
    arguments={"dataset_uri":"minio://mlpipeline/v2/artifacts/download-reddit/c1d12a96-5baf-410d-9e95-83333f005439/download-dataset/0dcdadc5-eb72-49ee-8e65-e4e7547eae86/dataset_path"}, 
    enable_caching=False)